In [1]:
import os
import glob
import altair as alt
import pandas as pd
import numpy as np
import warnings
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pathlib import Path
import os
import plotly
from tqdm import tqdm


pd.set_option('display.max_rows', 500)

In [2]:


print(path_base)

cols = ["Date", "Opponent", "Result", "W/L", "Div", "Spread", "Total", "Money"]
df_schedule = pd.read_csv(f"{path_base}/data/team_ranks/_nfl_team_{team_underscore}__{team_date}.csv")
df_schedule["Opponent"] = np.where(df_schedule.Location.eq("Away"), "@" + df_schedule.Opponent, df_schedule.Opponent)
record_str = df_schedule[~df_schedule.Div.isnull()].iloc[-1]["W/L"]

df_schedule = df_schedule.fillna("")
df_schedule = df_schedule.head(n=10)[cols]

df_schedule_opp = pd.read_csv(f"{path_base}/data/team_ranks/_nfl_team_{team_opp_underscore}__{team_date}.csv")
df_schedule_opp["Opponent"] = np.where(df_schedule_opp.Location.eq("Away"), "@" + df_schedule_opp.Opponent, df_schedule_opp.Opponent)
record_opp_str = df_schedule_opp[~df_schedule_opp.Div.isnull()].iloc[-1]["W/L"]

df_schedule_opp = df_schedule_opp.fillna("")
df_schedule_opp = df_schedule_opp.head(n=10)[cols]

file_path = f"{path_base}/data/team_ranks/_nfl_player_stat_*{team_date}.csv"
file_paths = glob.glob(os.path.join(file_path))

file_path = f"{path_base}/data/team_ranks/_nfl_stat*{nfl_date}.csv"
file_paths = glob.glob(os.path.join(file_path))
pds = list()

pbar = tqdm(file_paths)
for file_path in pbar:
    pbar.set_description(file_path)
    df = pd.read_csv(file_path)
    pds.append(df)
df = pd.concat(pds)

df["_stat"] = df.url.str.split("/", expand=True)[3]
df["Stat"] = df._stat.str.split("-").str.join(" ").str.title()
df["Stat"] = df.Stat.str.replace("Per Game", "Pg").str.replace("Opponent", "Opp")\
.str.replace("Offensive", "Off")\
.str.replace("Defensive", "Def")\
.str.replace("Touchdowns", "Td")\
.str.replace("Rushing", "Run")\
.str.replace("Passing", "Pass")\
.str.replace("Interceptions", "Int")

df["_team"] = df.Team.str.lower().str.split(" ").str.join("-")

# TODO do a second lookup with keys reversed
stat_matchup_map = {
    "points-per-game":"opponent-points-per-game",
    "points-per-play":"opponent-points-per-play",
    "passing-yards-per-game":"opponent-passing-yards-per-game",
    # sacks-per-game -> defense
    "qb-sacked-per-game":"sacks-per-game",
    "interceptions-per-game":"interceptions-thrown-per-game",
    "1st-half-points-per-game":"opponent-1st-half-points-per-game",
    "2nd-half-points-per-game":"opponent-2nd-half-points-per-game",
    "rushing-yards-per-game":"opponent-rushing-yards-per-game",
    "passing-yards-per-game":"opponent-passing-yards-per-game",
    "rushing-touchdowns-per-game":"opponent-rushing-touchdowns-per-game",
    "passing-touchdowns-per-game":"opponent-passing-touchdowns-per-game",
    "yards-per-game":"opponent-yards-per-game",
    "yards-per-play":"opponent-yards-per-play",
    "yards-per-pass-attempt":"opponent-yards-per-pass-attempt",
    "yards-per-rush-attempt":"opponent-yards-per-rush-attempt",
    "red-zone-scores-per-game":"opponent-red-zone-scores-per-game",
    "red-zone-scoring-pct":"opponent-zone-scoring-pct",
    "penalties-per-game":"opponent-penalties-per-game",
    "offensive-touchdowns-per-game":"opponent-offensive-touchdowns-per-game",
    "defensive-touchdowns-per-game":"opponent-defensive-touchdowns-per-game",
    "average-team-passer-rating":"opponent-average-team-passer-rating",
    "seconds-per-play":"opponent-seconds-per-play",
    "giveaways-per-game":"takeaways-per-game"
}
team_matchup_map = {"dallas":"green-bay"}

df = df.drop(columns=["Unnamed: 0"])


NameError: name 'path_base' is not defined

In [73]:
def find_matchup_rank(df, team_matchup_map, stat_matchup_map):
    df["matchup_key"] =  df._team +"-"+ df._stat
    df["matchup_team"] = df._team.map(team_matchup_map).str.title()
    df["matchup_key_lookup"] = df.matchup_team.str.lower() +"-"+ df._stat.map(stat_matchup_map)
    matchup_dict_data = {k:v for k,v in df.set_index("matchup_key")["Rank"].to_dict().items() if not pd.isna(k)}
    df["matchup_rank"] = df.matchup_key_lookup.map(matchup_dict_data)
    
    matchup_dict_data = {k:v for k,v in df.set_index("matchup_key")["2022"].to_dict().items() if not pd.isna(k)}
    df["matchup_value"] = df.matchup_key_lookup.map(matchup_dict_data)
    return df


In [74]:
#team_matchup_map.update({v:k for k,v in team_matchup_map.items()})
stat_matchup_map.update({v:k for k,v in stat_matchup_map.items()})

df = find_matchup_rank(df, team_matchup_map, stat_matchup_map)
 
#df = find_matchup_rank(df, {v:k for k,v in team_matchup_map.items()}, {v:k for k,v in stat_matchup_map.items()})

In [75]:
df["matchup_diff"] = abs(df.matchup_rank - df.Rank)

In [76]:
df_matchup = df[~df.matchup_rank.isna()][["Team", "matchup_team", "Stat", "2022", "Rank", "matchup_rank", "matchup_value", "matchup_diff"]].query("matchup_diff>=20").sort_values("Rank")


In [77]:
df_matchup

,Team,matchup_team,Stat,2022,Rank,matchup_rank,matchup_value,matchup_diff
1,Dallas,Green-Bay,Opp Points Per Play,0.253,2,27.0,0.271,25.0
1,Dallas,Green-Bay,Opp Off Td Pg,1.5,2,25.0,1.9,23.0
2,Dallas,Green-Bay,Opp Points Pg,16.6,3,27.0,17.1,24.0
2,Dallas,Green-Bay,Opp Red Zone Scores Pg,1.3,3,23.0,1.4,20.0
2,Dallas,Green-Bay,Opp 2Nd Half Points Pg,6.9,3,28.0,6.9,25.0
2,Dallas,Green-Bay,Giveaways Pg,0.9,3,27.0,0.9,24.0
3,Dallas,Green-Bay,Opp Run Td Pg,0.5,4,30.0,0.3,26.0
7,Dallas,Green-Bay,Seconds Per Play,27.9,8,30.0,30.8,22.0
24,Dallas,Green-Bay,Pass Yards Pg,190.6,25,2.0,170.6,23.0


In [14]:
chart_title = f""
x_cfgs = {"shorthand":"Rank", "title":"Chicago Rank"}
y_cfgs = {"shorthand":"Stat", "title":"Stat"}
y2_cfgs = {"shorthand":"matchup_rank", "title":"Detroit Rank"}

# background etc.
primary_color = "#0B162A" 
# font etc.
secondary_color = "#c83803"
axisLeftKwargs = dict(labelColor=secondary_color, titleColor=secondary_color, labelFontSize=15, titleFontSize=17)
axisBottomKwargs = dict(labelColor=secondary_color, titleColor=secondary_color,labelFontSize=15, titleFontSize=17)
legendKwargs = dict(labelColor=secondary_color, titleColor=secondary_color, strokeColor='gray', fillColor=primary_color, padding=10, cornerRadius=10,orient='top-right', labelFontSize=15, titleFontSize=15)

df

chart = alt.Chart(df_matchup, width=800, height=500, title=chart_title)\
.mark_bar()\
.encode(
    x=alt.X(**x_cfgs,),
    y=alt.Y(**y_cfgs,),
    x2="matchup_rank",
    #tooltip=[],

)

chart\
.configure(background=primary_color)\
.configure_axis(gridColor="grey", gridDash=[6,4])\
.configure_axisLeft(**axisLeftKwargs)\
.configure_axisBottom(**axisBottomKwargs)\
.configure_title(color=secondary_color, fontSize=25)\
.configure_legend(**legendKwargs)


alt.Chart(...)

In [27]:
# create a for loop that finds the opponent rank
# maype a function that sets the value of a column
# and creates a column called matchup rank
# should have something like a bad matchup count
# def find_matchup(home_team, away_team, stat):
# def apply find_matchup:=